### Join the beatport dataframes

In [2]:
import pandas as pd

In [ ]:
def load_beatport_data():
    """Load all Beatport CSV files into dataframes"""
    files = {
        'artists': 'data/bp_artist.csv',
        'artist_media': 'data/bp_artist_media.csv',
        'artist_release': 'data/bp_artist_release.csv',
        'artist_track': 'data/bp_artist_track.csv',
        'genres': 'data/bp_genre.csv',
        'keys': 'data/bp_key.csv',
        'label_artist': 'data/bp_label_artist.csv',
        'labels': 'data/bp_label.csv',
        'label_media': 'data/bp_label_media.csv',
        'releases': 'data/bp_release.csv',
        'release_media': 'data/bp_release_media.csv',
        'subgenres': 'data/bp_subgenre.csv',
        'tracks': 'data/bp_track.csv',
        'track_media': 'data/bp_track_media.csv'
}
    
    dfs = {}
    for key, file in files.items():
        dfs[key] = pd.read_csv(file)
    return dfs

def create_complete_dataset():
    """Create complete dataset by joining all Beatport data"""
    # Load all dataframes
    print("Loading data...")
    dfs = load_beatport_data()
    
    print("Starting joins...")
    
    # 1. Start with tracks and add track media
    print("Processing track information...")
    df = pd.merge(
        dfs['tracks'],
        dfs['track_media'],
        on='track_id',
        how='left',
        suffixes=('', '_track_media')
    )
    
    # 2. Add key information
    print("Adding key information...")
    df = pd.merge(
        df,
        dfs['keys'],
        on='key_id',
        how='left'
    )
    
    # 3. Add genre information
    print("Adding genre and subgenre information...")
    df = pd.merge(
        df,
        dfs['genres'],
        on='genre_id',
        how='left'
    )
    
    df = pd.merge(
        df,
        dfs['subgenres'],
        on=['genre_id', 'genre_url'],
        how='left',
        suffixes=('', '_subgenre')
    )
    
    # 4. Add release information
    print("Adding release information...")
    df = pd.merge(
        df,
        dfs['releases'],
        on=['release_id', 'label_id'],
        how='left',
        suffixes=('', '_release')
    )
    
    # 5. Add release media
    df = pd.merge(
        df,
        dfs['release_media'],
        on='release_id',
        how='left',
        suffixes=('', '_release_media')
    )
    
    # 6. Add label information
    print("Adding label information...")
    df = pd.merge(
        df,
        dfs['labels'],
        on='label_id',
        how='left',
        suffixes=('', '_label')
    )
    
    df = pd.merge(
        df,
        dfs['label_media'],
        on='label_id',
        how='left',
        suffixes=('', '_label_media')
    )
    
    # 7. Add artist relationships through artist_track
    print("Adding artist information...")
    df = pd.merge(
        df,
        dfs['artist_track'],
        on='track_id',
        how='left',
        suffixes=('', '_artist_track')
    )
    
    # 8. Add artist details
    df = pd.merge(
        df,
        dfs['artists'],
        on='artist_id',
        how='left',
        suffixes=('', '_artist')
    )
    
    # 9. Add artist media
    df = pd.merge(
        df,
        dfs['artist_media'],
        on='artist_id',
        how='left',
        suffixes=('', '_artist_media')
    )
    
    # Clean up columns
    print("Cleaning up the dataset...")
    # Remove duplicate updated_on columns
    columns_to_drop = [col for col in df.columns if col.startswith('updated_on_')]
    # Remove duplicate URL columns and other duplicates
    columns_to_drop.extend([col for col in df.columns if col.endswith(('_x', '_y'))])
    df = df.drop(columns=columns_to_drop)
    
    return df

def generate_dataset_report(df):
    """Generate a report about the dataset"""
    print("\nDataset Report:")
    print("-" * 50)
    print(f"Total number of rows: {len(df)}")
    print(f"Total number of columns: {len(df.columns)}")
    print("\nUnique counts:")
    print(f"Tracks: {df['track_id'].nunique()}")
    print(f"Artists: {df['artist_id'].nunique()}")
    print(f"Labels: {df['label_id'].nunique()}")
    print(f"Releases: {df['release_id'].nunique()}")
    print(f"Genres: {df['genre_id'].nunique()}")
    print(f"Subgenres: {df['subgenre_id'].nunique()}")
    print(f"Keys: {df['key_id'].nunique()}")
    
    print("\nMissing values summary:")
    missing = df.isnull().sum()
    missing = missing[missing > 0]
    print(missing)

def main():
    # Create the complete dataset
    print("Starting data integration process...")
    final_df = create_complete_dataset()
    
    # Generate and display report
    generate_dataset_report(final_df)
    
    # Save the final dataset
    print("\nSaving complete dataset...")
    final_df.to_csv('beatport_complete_data.csv', index=False)
    print("Dataset saved successfully!")
    
    return final_df

if __name__ == "__main__":
    final_df = main()

Starting data integration process...
Loading data...


C:\Users\olivi\AppData\Local\Temp\ipykernel_24680\1614804988.py:24: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  dfs[key] = pd.read_csv(file)


Starting joins...
Processing track information...
Adding key information...
Adding genre and subgenre information...
Adding release information...
Adding label information...
Adding artist information...
Cleaning up the dataset...

Dataset Report:
--------------------------------------------------
Total number of rows: 36900320
Total number of columns: 52

Unique counts:
Tracks: 10685331
Artists: 824763
Labels: 78283
Releases: 2598492
Genres: 32
Subgenres: 72
Keys: 34

Missing values summary:
title                        157
mix                            9
subgenre_id             32031014
duration                   10062
duration_ms                10062
isrc                      914141
key_id                     20539
is_matched_spot         12760617
wave_img_id                55648
wave_img_uuid              55648
sample_start               10696
sample_end                 10696
key_letter                 20539
key_name                   20539
camelot_num                20539
camelot

In [1]:

import pandas as pd
import os

# Create output directory
output_dir = "split_by_genre"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Dictionary to track files
genre_files = {}
chunk_number = 0

# Process the file in chunks
print("Starting to process file in chunks...")
for chunk in pd.read_csv("beatport_complete_data.csv", chunksize=100000):
    chunk_number += 1
    print(f"Processing chunk {chunk_number}...")
    
    # Process each genre in the chunk
    for genre in chunk['genre_name'].unique():
        # Create safe filename
        safe_genre = "".join(c if c.isalnum() or c in (' ', '-') else '_' for c in genre).rstrip()
        filename = os.path.join(output_dir, f"beatport_{safe_genre}.csv")
        
        # Get rows for this genre
        genre_chunk = chunk[chunk['genre_name'] == genre]
        
        # Append or create new file
        if genre in genre_files:
            genre_chunk.to_csv(filename, mode='a', header=False, index=False)
        else:
            genre_chunk.to_csv(filename, mode='w', header=True, index=False)
            genre_files[genre] = filename
            print(f"Created new file for genre: {genre}")
    
    # Clear memory
    del chunk

print("\nFinished processing!")
print(f"Split into {len(genre_files)} files:")
for genre, filepath in genre_files.items():
    file_size = os.path.getsize(filepath) / (1024 * 1024)  # Convert to MB
    print(f"- {genre}: {file_size:.2f} MB")


Starting to process file in chunks...


C:\Users\olivi\AppData\Local\Temp\ipykernel_27380\3690345104.py:15: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv("beatport_complete_data.csv", chunksize=100000):


Processing chunk 1...
Created new file for genre: Trance (Raw / Deep / Hypnotic)
Created new file for genre: Techno (Raw / Deep / Hypnotic)
Created new file for genre: Tech House
Created new file for genre: Amapiano
Created new file for genre: Deep House
Created new file for genre: Electro (Classic / Detroit / Modern)
Created new file for genre: 140 / Deep Dubstep / Grime
Created new file for genre: Bass / Club
Created new file for genre: Jackin House
Created new file for genre: UK Garage / Bassline


C:\Users\olivi\AppData\Local\Temp\ipykernel_27380\3690345104.py:15: DtypeWarning: Columns (15,33,35,36) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv("beatport_complete_data.csv", chunksize=100000):


Processing chunk 2...
Created new file for genre: Bass House
Created new file for genre: Afro House
Created new file for genre: Hard Techno


...

Processing chunk 310...
Processing chunk 311...
Processing chunk 312...
Processing chunk 313...
Processing chunk 314...
Processing chunk 315...
Processing chunk 316...
Processing chunk 317...
Processing chunk 318...
Processing chunk 319...
Processing chunk 320...
Processing chunk 321...
Processing chunk 322...
Processing chunk 323...
Processing chunk 324...
Processing chunk 325...
Processing chunk 326...
Processing chunk 327...
Processing chunk 328...
Processing chunk 329...
Processing chunk 330...
Processing chunk 331...
Processing chunk 332...
Processing chunk 333...
Processing chunk 334...


C:\Users\olivi\AppData\Local\Temp\ipykernel_27380\3690345104.py:15: DtypeWarning: Columns (33,35,36) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv("beatport_complete_data.csv", chunksize=100000):


Processing chunk 335...


C:\Users\olivi\AppData\Local\Temp\ipykernel_27380\3690345104.py:15: DtypeWarning: Columns (33,35,36) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv("beatport_complete_data.csv", chunksize=100000):


Processing chunk 336...
Processing chunk 337...
Processing chunk 338...
Processing chunk 339...
Processing chunk 340...
Processing chunk 341...
Processing chunk 342...
Processing chunk 343...
Processing chunk 344...
Processing chunk 345...
Processing chunk 346...
Processing chunk 347...
Processing chunk 348...
Processing chunk 349...
Processing chunk 350...
Processing chunk 351...
Processing chunk 352...
Processing chunk 353...
Processing chunk 354...
Processing chunk 355...
Processing chunk 356...
Processing chunk 357...
Processing chunk 358...
Processing chunk 359...
Processing chunk 360...
Processing chunk 361...
Processing chunk 362...
Processing chunk 363...
Processing chunk 364...
Processing chunk 365...
Processing chunk 366...
Processing chunk 367...
Processing chunk 368...
Processing chunk 369...
Processing chunk 370...

Finished processing!
Split into 32 files:
- Trance (Raw / Deep / Hypnotic): 5.27 MB
- Techno (Raw / Deep / Hypnotic): 524.03 MB
- Tech House: 927.32 MB
- Amapi

In [3]:
techno_raw_deep_hypnotic_df = pd.read_csv("split_by_genre/beatport_techno_raw_deep_hypnotic.csv")

C:\Users\olivi\AppData\Local\Temp\ipykernel_43640\240625614.py:1: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  techno_raw_deep_hypnotic_df = pd.read_csv("split_by_genre/beatport_techno_raw_deep_hypnotic.csv")


In [7]:
techno_raw_deep_hypnotic_df.head(5)

,track_id,title,mix,is_remixed,release_date,genre_id,subgenre_id,track_url,bpm,duration,...,label_name,label_url,label_img_id,label_img_uuid,artist_id,is_remixer,artist_name,artist_url,artist_img_id,artist_img_uuid
0,387142,Avion,Original Mix,f,1994-06-22,92,NaN,beatport.com/track/avion/387142,130,5:52,...,Synewave,beatport.com/label/synewave/2587,3231998,0d3916f7-e37e-4514-a952-a0187452096f,8004,f,Damon Wild,beatport.com/artist/damon-wild/8004,245396,48f7f2f3-cc67-450f-9e47-1c0269dac7ae
1,387142,Avion,Original Mix,f,1994-06-22,92,NaN,beatport.com/track/avion/387142,130,5:52,...,Synewave,beatport.com/label/synewave/2587,3231998,0d3916f7-e37e-4514-a952-a0187452096f,8004,f,Damon Wild,beatport.com/artist/damon-wild/8004,245396,48f7f2f3-cc67-450f-9e47-1c0269dac7ae
2,387142,Avion,Original Mix,f,1994-06-22,92,NaN,beatport.com/track/avion/387142,130,5:52,...,Synewave,beatport.com/label/synewave/2587,3231998,0d3916f7-e37e-4514-a952-a0187452096f,8004,f,Damon Wild,beatport.com/artist/damon-wild/8004,245396,48f7f2f3-cc67-450f-9e47-1c0269dac7ae
3,387142,Avion,Original Mix,f,1994-06-22,92,NaN,beatport.com/track/avion/387142,130,5:52,...,Synewave,beatport.com/label/synewave/2587,3231998,0d3916f7-e37e-4514-a952-a0187452096f,8004,f,Damon Wild,beatport.com/artist/damon-wild/8004,245396,48f7f2f3-cc67-450f-9e47-1c0269dac7ae
4,387142,Avion,Original Mix,f,1994-06-22,92,NaN,beatport.com/track/avion/387142,130,5:52,...,Synewave,beatport.com/label/synewave/2587,3231998,0d3916f7-e37e-4514-a952-a0187452096f,8004,f,Damon Wild,beatport.com/artist/damon-wild/8004,245396,48f7f2f3-cc67-450f-9e47-1c0269dac7ae


In [4]:
techno_raw_deep_hypnotic_df.columns

Index(['track_id', 'title', 'mix', 'is_remixed', 'release_date', 'genre_id',
       'subgenre_id', 'track_url', 'bpm', 'duration', 'duration_ms', 'isrc',
       'key_id', 'label_id', 'release_id', 'is_matched_spot', 'wave_img_id',
       'wave_img_uuid', 'sample_uuid', 'sample_start', 'sample_end',
       'key_letter', 'key_name', 'camelot_num', 'camelot_letter', 'is_sharp',
       'is_flat', 'chord_id', 'chord_name', 'genre_name', 'song_count',
       'genre_url', 'subgenre_id_subgenre', 'subgenre_name',
       'song_count_subgenre', 'subgenre_url', 'updated_on', 'release_title',
       'release_date_release', 'release_url', 'release_img_id',
       'release_img_uuid', 'label_name', 'label_url', 'label_img_id',
       'label_img_uuid', 'artist_id', 'is_remixer', 'artist_name',
       'artist_url', 'artist_img_id', 'artist_img_uuid'],
      dtype='object')

In [9]:
techno_raw_deep_hypnotic_df["is_matched_spot"].dtypes


dtype('O')

In [10]:
def remove_sub_genre(filename):
    df = pd.read_csv(filename)
    df = df.drop(columns=['subgenre_id', 'subgenre_id_subgenre', 'subgenre_name', 'song_count_subgenre', 'subgenre_url', 'updated_on', "is_matched_spot"])
    df = df.drop_duplicates(subset=['track_id'])
    df.to_csv(filename, '_no_subgenre', index=False)


In [11]:
import pandas as pd
import os

def remove_sub_genre(filename):
    df = pd.read_csv(filename)
    df = df.drop(columns=['subgenre_id', 'subgenre_id_subgenre', 'subgenre_name', 'song_count_subgenre', 'subgenre_url', 'updated_on', "is_matched_spot"])
    df = df.drop_duplicates(subset=['track_id'])
    # Fix the output filename creation - using os.path.join and string formatting
    output_filename = os.path.join(os.path.dirname(filename), f"{os.path.splitext(os.path.basename(filename))[0]}_no_subgenre.csv")
    df.to_csv(output_filename, index=False)
    print(f"Processed {filename} -> {output_filename}")

# Path to your folder
folder_path = "split_by_genre"

# Iterate through all CSV files in the folder
for filename in os.listdir(folder_path):
    if filename.endswith(".csv"):
        full_path = os.path.join(folder_path, filename)
        remove_sub_genre(full_path)

C:\Users\olivi\AppData\Local\Temp\ipykernel_43640\3700072806.py:5: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filename)


Processed split_by_genre\beatport_afro_house.csv -> split_by_genre\beatport_afro_house_no_subgenre.csv
Processed split_by_genre\beatport_amapiano.csv -> split_by_genre\beatport_amapiano_no_subgenre.csv
Processed split_by_genre\beatport_Bass House.csv -> split_by_genre\beatport_Bass House_no_subgenre.csv


C:\Users\olivi\AppData\Local\Temp\ipykernel_43640\3700072806.py:5: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filename)


Processed split_by_genre\beatport_bass_club.csv -> split_by_genre\beatport_bass_club_no_subgenre.csv


C:\Users\olivi\AppData\Local\Temp\ipykernel_43640\3700072806.py:5: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filename)


...

Processed split_by_genre\beatport_trance.csv -> split_by_genre\beatport_trance_no_subgenre.csv
Processed split_by_genre\beatport_trance_raw_deep_hypnotic_.csv -> split_by_genre\beatport_trance_raw_deep_hypnotic__no_subgenre.csv


C:\Users\olivi\AppData\Local\Temp\ipykernel_43640\3700072806.py:5: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filename)


Processed split_by_genre\beatport_trap_wave.csv -> split_by_genre\beatport_trap_wave_no_subgenre.csv


C:\Users\olivi\AppData\Local\Temp\ipykernel_43640\3700072806.py:5: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filename)


Processed split_by_genre\beatport_uk_garage_bassline.csv -> split_by_genre\beatport_uk_garage_bassline_no_subgenre.csv


In [15]:
techno_raw_deep_hypnotic_no_subgenre_df = pd.read_csv('split_by_genre_no_subgenre/beatport_techno_raw_deep_hypnotic_no_subgenre.csv')
techno_raw_deep_hypnotic_no_subgenre_df

,track_id,title,mix,is_remixed,release_date,genre_id,track_url,bpm,duration,duration_ms,...,label_name,label_url,label_img_id,label_img_uuid,artist_id,is_remixer,artist_name,artist_url,artist_img_id,artist_img_uuid
0,387142,Avion,Original Mix,f,1994-06-22,92,beatport.com/track/avion/387142,130,5:52,352466.0,...,Synewave,beatport.com/label/synewave/2587,3231998,0d3916f7-e37e-4514-a952-a0187452096f,8004,f,Damon Wild,beatport.com/artist/damon-wild/8004,245396,48f7f2f3-cc67-450f-9e47-1c0269dac7ae
1,473092,Rotary,Regis Mix,t,1994-06-22,92,beatport.com/track/rotary/473092,131,4:15,255012.0,...,Synewave,beatport.com/label/synewave/2587,3231998,0d3916f7-e37e-4514-a952-a0187452096f,8004,f,Damon Wild,beatport.com/artist/damon-wild/8004,245396,48f7f2f3-cc67-450f-9e47-1c0269dac7ae
2,3402856,001#5,Original Mix,f,2012-03-30,92,beatport.com/track/0015/3402856,116,7:44,464318.0,...,Rawax,beatport.com/label/rawax/22353,3942313,8c92d23c-54f7-4d27-a380-c8e9d73ed9d2,256549,f,Vualitron,beatport.com/artist/vualitron/256549,5539565,0dc61986-bccf-49d4-8fad-6b147ea8f327
3,3402852,001#1,Original Mix,f,2012-03-30,92,beatport.com/track/0011/3402852,116,6:25,385100.0,...,Rawax,beatport.com/label/rawax/22353,3942313,8c92d23c-54f7-4d27-a380-c8e9d73ed9d2,256549,f,Vualitron,beatport.com/artist/vualitron/256549,5539565,0dc61986-bccf-49d4-8fad-6b147ea8f327
4,3332498,Czt,Original Mix,f,2012-03-02,92,beatport.com/track/czt/3332498,125,11:15,675840.0,...,Silent Steps,beatport.com/label/silent-steps/18855,10060408,5dcc3ced-d42f-4a3a-9a35-5e8479ee664a,132943,f,The Noisemaker,beatport.com/artist/the-noisemaker/132943,6484800,7d660daa-a9aa-4ede-b92f-84bb1c18279c
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125386,649022,Bienfait 2,Original Mix,f,2000-05-24,92,beatport.com/track/bienfait-2/649022,71,5:31,331386.0,...,Kompakt,beatport.com/label/kompakt/4530,11537448,2a4c2937-e97c-40a7-8be8-605c7a9b5169,4984,f,Jonas Bering,beatport.com/artist/jonas-bering/4984,12330837,7cb378b7-c612-48aa-82a2-15a86549ebd1
125387,5635314,Prowler,Original Mix,f,2001-12-17,92,beatport.com/track/prowler/5635314,67,7:40,460846.0,...,Counterbalance,beatport.com/label/counterbalance/119,324,af7e018a-3213-44c7-801c-8597077cc75b,4036,f,Surgeon,beatport.com/artist/surgeon/4036,42975,0c86b134-ccc0-4268-b7f9-e4fadd9cc00a
125388,5635344,"La Real, Pt. 1",Original Mix,f,2000-03-13,92,beatport.com/track/la-real-pt-1/5635344,134,6:12,372700.0,...,Counterbalance,beatport.com/label/counterbalance/119,324,af7e018a-3213-44c7-801c-8597077cc75b,4036,f,Surgeon,beatport.com/artist/surgeon/4036,42975,0c86b134-ccc0-4268-b7f9-e4fadd9cc00a
125389,17248613,Blow That Shit Out,Original Mix,f,1995-02-02,92,beatport.com/track/blow-that-shit-out/17248613,135,3:40,220800.0,...,Traxmen Records,beatport.com/label/traxmen-records/41074,9469993,2eb45127-56ea-4d36-a077-2f9eff0dcd13,6374,f,Robert Armani,beatport.com/artist/robert-armani/6374,31603800,99dde20c-2ece-4d40-a857-b286d0102b1a


In [16]:
df_complete_no_subgenre = pd.read_csv("beatport_complete_data.csv")
df_complete_no_subgenre = df_complete_no_subgenre.drop(columns=['subgenre_id', 'subgenre_id_subgenre', 'subgenre_name', 'song_count_subgenre', 'subgenre_url', 'updated_on', "is_matched_spot"])
df_complete_no_subgenre = df_complete_no_subgenre.drop_duplicates(subset=['track_id'])
df_complete_no_subgenre.to_csv("beatport_complete_data_no_subgenre.csv", index=False)

C:\Users\olivi\AppData\Local\Temp\ipykernel_43640\3487171811.py:1: DtypeWarning: Columns (15,33,35,36) have mixed types. Specify dtype option on import or set low_memory=False.
  df_complete_no_subgenre = pd.read_csv("beatport_complete_data.csv")


In [18]:
df_complete_no_subgenre.head(50).to_csv('beatport_complete_no_subgenre_sample.csv')

In [19]:
df_complete_spotify = pd.read_csv("complete/spotify_complete_data.csv")
df_complete_spotify.head(50).to_csv('complete/spotify_complete_sample.csv')
